In [ ]:
!pip install implicit -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 58.8 MB/s eta 0:00:00


In [16]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import scipy
from implicit.nearest_neighbours import bm25_weight
import implicit

In [ ]:
def apk(actual, predicted, k=3):

    if actual is None or len(actual) == 0:
        return 0.0

    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        # first condition checks whether it is valid prediction
        # second condition checks if prediction is not repeated
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    return score / min(len(actual), k)

In [ ]:
def mapk(actual, predicted, k=3):

    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

In [ ]:
def get_date_k_weeks_before(cur_date: str, k: int = 2) -> str:

    cur_date = datetime.datetime.strptime(cur_date, "%Y-%m-%d")
    return (cur_date - datetime.timedelta(weeks=k)).strftime("%Y-%m-%d")

In [ ]:
def remove_duplicates_order(x: list) -> list:

    return list(dict.fromkeys(x))


In [ ]:
def get_validation_split(
    df: pd.DataFrame, timestamp_to_split: str
) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    df = df.sort_values("timestamp", ascending=False)
    data_train = df[df["timestamp"] < timestamp_to_split]
    val = df[df["timestamp"] >= timestamp_to_split]

    gt_val = val.groupby("user_id")[["product_id"]].agg(list).reset_index()
    gt_val["product_id"] = gt_val["product_id"].apply(remove_duplicates_order)

    timestamp_to_val_split = get_date_k_weeks_before(timestamp_to_split, k=2)
    data_val = data_train[data_train["timestamp"] >= timestamp_to_val_split]
    data_train = data_train[
        data_train["timestamp"] < timestamp_to_val_split
    ].reset_index(drop=True)

    gt_val = gt_val[gt_val["user_id"].isin(set(data_val["user_id"]))].reset_index(
        drop=True
    )
    data_val = data_val[data_val["user_id"].isin(set(gt_val["user_id"]))].reset_index(
        drop=True
    )

    gt_val_val = data_val.groupby("user_id")[["product_id"]].agg(list).reset_index()

    merged = gt_val_val.merge(gt_val, on="user_id")

    changed = []
    for idx in merged.index:
        prev, fut = merged.iloc[idx, 1:]
        if isinstance(prev, int):
            prev = [prev]
        if isinstance(fut, int):
            fut = [fut]
        out = list(set(fut).difference(set(prev)))
        if len(out) > 0:
            changed.append(out)
        else:
            changed.append(np.NAN)

    merged["product_id"] = changed

    merged = merged.dropna()

    return (
        data_train,
        data_val,
        merged[["user_id", "product_id"]],
        merged[["user_id", "product_id_x"]],
    )

In [15]:
data = pd.read_parquet("/content/drive/MyDrive/train.parquet")
data_final_test = pd.read_parquet("/content/drive/MyDrive/test.parquet")
data["price"] = data["price"].astype(float)
data_final_test["price"] = data_final_test["price"].astype(float)

In [17]:
# Учим на данных с февраля так как там не менялись названия и категории товаров
data_feb = data.loc[data["timestamp"] >= "2020-02-01"]

data_feb["timestamp"].quantile(0.5, interpolation="nearest")

Timestamp('2020-02-14 07:48:09')

In [18]:
data_train, data_val, gt_val, gt_val_val = get_validation_split(
    df=data_feb, timestamp_to_split="2020-02-14"
)
gt_val_val.columns = ["user_id", "product_id"]


In [19]:
data_grouped = (
    data_val.groupby(["user_id", "product_id"])[["timestamp"]]
    .count()
    .reset_index()
    .rename(columns={"timestamp": "count"})
)
data_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106360 entries, 0 to 106359
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   user_id     106360 non-null  int64
 1   product_id  106360 non-null  int64
 2   count       106360 non-null  int64
dtypes: int64(3)
memory usage: 2.4 MB


In [20]:
prd_set = set(data_grouped["product_id"])
print(f"{len(prd_set)} unique product_id")
prd_dict = {url: idurl for url, idurl in zip(prd_set, range(len(prd_set)))}
usr_set = set(data_grouped["user_id"])
print(f"{len(usr_set)} unique users")
usr_dict = {usr: user_id for usr, user_id in zip(usr_set, range(len(usr_set)))}

18149 unique product_id
47674 unique users


In [21]:
values = np.array(data_grouped["count"])
rows = np.array(data_grouped["user_id"].map(usr_dict))
cols = np.array(data_grouped["product_id"].map(prd_dict))
mat = scipy.sparse.coo_matrix(
    (values, (rows, cols)), shape=(rows.max() + 1, cols.max() + 1)
)

In [25]:
user_product = bm25_weight(mat, K1=100, B=0.8)

# get the transpose since the most of the functions in implicit expect (user, item) sparse matrices instead of (item, user)
user_product = user_product.tocsr()
user_plays = user_product

In [26]:
als = implicit.als.AlternatingLeastSquares(
    factors=16, regularization=1, alpha=3, iterations=100, random_state=42
)

In [27]:
als.fit(user_product)

  0%|          | 0/100 [00:00<?, ?it/s]

In [28]:
# Идея в том, чтобы в для юзеров с низким score предлагать 3 самых популярных из тех, что они не брали
preds = []
print(gt_val["user_id"].shape[0])

popular_from_train = (
    data_val.groupby(["product_id"])[["event_time"]]
    .count()
    .reset_index()
    .sort_values(by="event_time", ascending=False)
    .rename(columns={"event_time": "count"})
).reset_index(drop=True)

for i, user in tqdm(enumerate(gt_val["user_id"])):
    userid = usr_dict[user]
    ids, scores = als.recommend(
        userid, user_plays[userid], N=3, filter_already_liked_items=True
    )
    if scores[0] > 0.3:
        preds.append(list(np.array(list(prd_dict.keys()))[ids]))
    else:
        prod = gt_val_val.iloc[i, 1]
        preds.append(
            popular_from_train[~popular_from_train["product_id"].isin(prod)]
            .head(3)["product_id"]
            .to_list()
        )

40076


0it [00:00, ?it/s]

In [29]:
# 0.07925
round(
    mapk(
        actual=gt_val["product_id"].to_list(),
        predicted=preds,
        k=3,
    ),
    5,
)

0.07925

In [30]:
data_grouped = (
    data_final_test.groupby(["user_id", "product_id"])[["timestamp"]]
    .count()
    .reset_index()
    .rename(columns={"timestamp": "count"})
)
data_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86074 entries, 0 to 86073
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   user_id     86074 non-null  int64
 1   product_id  86074 non-null  int64
 2   count       86074 non-null  int64
dtypes: int64(3)
memory usage: 2.0 MB


In [31]:
prd_set = set(data_grouped["product_id"])
print(f"{len(prd_set)} unique product_id")
prd_dict = {url: idurl for url, idurl in zip(prd_set, range(len(prd_set)))}
usr_set = set(data_grouped["user_id"])
print(f"{len(usr_set)} unique users")
usr_dict = {usr: user_id for usr, user_id in zip(usr_set, range(len(usr_set)))}

15478 unique product_id
36169 unique users


In [32]:
values = np.array(data_grouped["count"])
rows = np.array(data_grouped["user_id"].map(usr_dict))
cols = np.array(data_grouped["product_id"].map(prd_dict))
mat = scipy.sparse.coo_matrix(
    (values, (rows, cols)), shape=(rows.max() + 1, cols.max() + 1)
)

In [36]:
valid_ids = [idx for idx in ids if idx < len(prd_dict.keys())]



In [37]:
gt_val_test = data_final_test.groupby("user_id")[["product_id"]].agg(list).reset_index()
gt_val_test["product_id"] = gt_val_test["product_id"].apply(remove_duplicates_order)


preds = []

popular_from_train = (
    data_final_test.groupby(["product_id"])[["event_time"]]
    .count()
    .reset_index()
    .sort_values(by="event_time", ascending=False)
    .rename(columns={"event_time": "count"})
).reset_index(drop=True)

for i, user in enumerate(gt_val_test["user_id"]):
    userid = usr_dict[user]
    ids, scores = als.recommend(
        userid, user_plays[userid], N=3, filter_already_liked_items=True
    )
    if scores[0] > 0.3:
        preds.append(list(np.array(list(prd_dict.keys()))[valid_ids]))
    else:
        prod = gt_val_test.iloc[i, 1]
        preds.append(
            popular_from_train[~popular_from_train["product_id"].isin(prod)]
            .head(3)["product_id"]
            .to_list()
        )

In [38]:
df_sub = pd.DataFrame()
df_sub["user_id"] = gt_val_test["user_id"]
df_sub["product_id"] = preds
df_sub["product_id"] = df_sub["product_id"].apply(lambda x: " ".join(map(str, x)))
df_sub

,user_id,product_id
0,313268536,
1,420986446,
2,434350078,
3,440520758,
4,457018572,
...,...,...
36164,628852419,
36165,628857344,
36166,628858687,
36167,628865126,1005115 1005100 1004767


In [39]:
df_sub.to_csv("sample_submission.csv", index=False)